In [53]:
import pandas as pd
import pickle
import re
from collections import defaultdict

In [7]:
census_blocks_df = pd.read_csv('../data/block_shapes/CensusBlockTIGER2010.csv')

In [115]:
(census_blocks_df)
# census_blocks_df.shape
census_blocks.TRACT_BLOC.count()


46357

In [18]:
block_dict = {}
for index, row in census_blocks_df.iterrows():
    block_coord = row['the_geom'].split(" ")
    lat_pat = re.compile('[^-\d][\d]+[.][\d]+')
    lats = lat_pat.findall(str(block_coord))
    long_pat = re.compile('[-][\d]+[.][\d]+')
    longs = long_pat.findall(str(block_coord))
    float_lats = [float(x[1:]) for x in lats]
    avg_lat = sum(float_lats)/len(float_lats)
    float_longs = [float(x) for x in longs]
    avg_long = sum(float_longs)/len(float_longs)

    block_dict[row['NAME10']] = [avg_lat, avg_long]

In [99]:
len(block_dict)



651

## Coffee Shops


In [111]:
import math

In [76]:
with open('../data/yelp/yelp_coffee_shop_dict.pk', 'rb') as read_file:
    coffee_shops = pickle.load(read_file)

In [77]:
block_ex = block_dict['Block 4007']
shop_ex = coffee_shops['f5p7scO1Q9UdaGbYe9bjFg']

In [78]:
block_coffee_dict = defaultdict(int)
for key_b, value_b in block_dict.items():
    for key_c, value_c in coffee_shops.items():
        lat_block = value_b[0]
        lon_block = value_b[1]
        lat_shop = value_c[2]
        lon_shop = value_c[3]
        dist = math.sqrt((lat_block -lat_shop)**2 + (lon_block - lon_shop)**2)
        if dist < .035:
            block_coffee_dict[key_b] += 1


In [98]:
#Here are the coffeeshops within a 2 mile radius of each block
len(block_coffee_dict)


514

In [51]:
dist = math.sqrt((41.963673-41.964346)**2 + (-87.786847 - -87.751445)**2)

In [52]:
dist

0.03540839636300031

lat_long distance of .035 is about 2 miles

## Businesses

In [59]:
df_businesses = pd.read_csv('../data/Business_Licenses_-_Current_Active.csv')

In [68]:
df_businesses.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,2068918-20170316,2509609,358113,1,"AG 26TH, INC.",SHELL,3201 W 26TH ST 1ST,CHICAGO,IL,60623,...,03/16/2017,03/15/2019,01/16/2017,01/17/2017,AAI,NaN,25.0,41.844407,-87.705100,"(41.844407330022506, -87.70509984940453)"
1,2507033-20170117,2507033,281848,3,"DRIEHAUS ENTERPRISE MANAGEMENT, INC.",Driehaus Enterprise Management Inc,737 N MICHIGAN AVE 2220,CHICAGO,IL,60611,...,01/17/2017,03/15/2019,01/17/2017,01/17/2017,AAI,NaN,NaN,41.896036,-87.624131,"(41.89603605011911, -87.62413103273497)"
2,2068924-20170316,2509610,358115,1,"AG NORTH AVENUE BEACH, INC.",SHELL,130 W NORTH AVE 1ST,CHICAGO,IL,60610,...,03/16/2017,03/15/2019,01/16/2017,01/17/2017,AAI,NaN,NaN,41.911272,-87.633117,"(41.91127206809849, -87.63311717438927)"
3,2487557-20161220,2487557,374871,3,SUPREME BEAUTY PARLOR LLC,SUPREME BEAUTY PARLOR LLC,2944 W MONTROSE AVE,CHICAGO,IL,60618,...,12/20/2016,03/15/2019,12/20/2016,12/20/2016,AAI,NaN,60.0,41.961333,-87.702711,"(41.96133298173611, -87.70271111999868)"
4,2506708-20170106,2506708,304284,3,THE IT ARCHITECT CORPORATION,THE IT ARCHITECT CORPORATION,505 N LAKE SHORE DR 2ND 217,CHICAGO,IL,60611,...,01/06/2017,03/15/2019,01/06/2017,01/06/2017,AAI,NaN,NaN,41.891279,-87.614108,"(41.89127929349215, -87.61410753333017)"


In [69]:
business_dict = {}
for i,row in df_businesses.iterrows():
    business_dict[row['LICENSE ID']] = [row.LATITUDE, row.LONGITUDE]

In [72]:
business_dict[2507289][0]

41.901384635

In [80]:
block_bus_dict = defaultdict(int)
for key_b, value_b in block_dict.items():
    for key_bus, value_bus in business_dict.items():
        lat_block = value_b[0]
        lon_block = value_b[1]
        lat_bus = value_bus[0]
        lon_bus = value_bus[1]
        dist = math.sqrt((lat_block -lat_bus)**2 + (lon_block - lon_bus)**2)
        if dist < .035:
            block_bus_dict[key_b] += 1

In [97]:
len(block_bus_dict)




651

## Merge coffee shop count and business count

In [83]:
block_bus_df = pd.DataFrame.from_dict(block_bus_dict, orient='index')

In [89]:
block_bus_df.rename(columns = {0:'business_count'}, inplace=True)

In [92]:
block_coffee_df = pd.DataFrame.from_dict(block_coffee_dict, orient='index')
block_coffee_df.rename(columns = {0:'coffeeshop_count'}, inplace=True)

In [94]:
df_block_bus = pd.merge(block_coffee_df, block_bus_df, left_index=True, right_index=True)

In [96]:
df_block_bus
df_block_bus.shape



(514, 2)